In [14]:
import torch
import torch.nn as nn
from sentiment_preprocessing import SentimentAnalysisDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import sys
sys.path.append('..')
from ELMO import ELMo

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
path = 'Marathi_Semantic/pos_train.txt'
with open (path, "r", encoding='utf-8') as f:
    pos_train = f.readlines()
path = 'Marathi_Semantic/neg_train.txt'
with open (path, "r", encoding='utf-8') as f:
    neg_train = f.readlines()

In [6]:
# make a tensor of labels
y_pos = torch.ones(len(pos_train), dtype=torch.long)
y_neg = torch.zeros(len(neg_train), dtype=torch.long)
labels = torch.cat((y_pos, y_neg), dim=0)

In [7]:
data = pos_train + neg_train

In [8]:
char_vocab = torch.load('../ELMo/char_vocab_marathi.pt')
word_vocab = torch.load('../ELMo/word_vocab_marathi.pt')

In [9]:
elmo = ELMo(cnn_config = {'character_embedding_size': 16, 
                           'num_filters': 32, 
                           'kernel_size': 5, 
                           'max_word_length': 10, 
                           'char_vocab_size': char_vocab.num_chars}, 
             elmo_config = {'num_layers': 3,
                            'word_embedding_dim': 150,
                            'vocab_size': word_vocab.num_words}, 
             char_vocab_size = char_vocab.num_chars).to(device)

In [10]:
# load model
elmo.load_state_dict(torch.load('../ELMo/elmo_epoch_3.pt'))

<All keys matched successfully>

In [11]:
dataset = SentimentAnalysisDataset(data[:2], labels, word_vocab, char_vocab)
# split into train and val
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=dataset.collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [12]:
class SentimentAnalysis(nn.Module):
    def __init__(self, elmo, embedding_dim, num_classes):
        super(SentimentAnalysis, self).__init__()
        self.elmo = elmo
        self.lambdas = nn.Parameter(torch.randn(3))
        self.lstm = nn.LSTM(embedding_dim, embedding_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.fc = nn.Linear(embedding_dim, embedding_dim//2)
        self.fc2 = nn.Linear(embedding_dim//2, num_classes)
        self.relu = nn.ReLU()

        for param in self.elmo.parameters():
            param.requires_grad = False

    def forward(self, x):
        forward_output, backward_output, final_embeddings = self.elmo(x)
        encoding = torch.zeros_like(final_embeddings[0])
        for i in range(3):
            encoding += self.lambdas[i] * final_embeddings[i]
        output, (hidden, cell) = self.lstm(encoding)
        final_output = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        output = self.fc(final_output)
        output = self.relu(output)
        output = self.fc2(output)
        return output

In [13]:
sentiment_analysis = SentimentAnalysis(elmo, 300, 2).to(device)
optimizer = torch.optim.Adam(sentiment_analysis.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_sentiment(model, train_loader, val_loader, optimizer, criterion, epochs):
    model.train()
    model.to(device)
    for epoch in range(epochs):
        total_loss = 0
        predictions = []
        targets = []
        for batch in tqdm(train_dataloader)
    